# Coletando dados de temperatura de uma API

## Sobre os dados
Neste notebook, estaremos coletando dados diários de temperatura da [API dos Centros Nacionais de Informação Ambiental (NCEI)](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). Utilizaremos o conjunto de dados Global Historical Climatology Network - Daily (GHCND); veja a documentação [aqui](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

*Nota: A NCEI faz parte da Administração Nacional Oceânica e Atmosférica (NOAA) e, como você pode ver no URL da API, este recurso foi criado quando a NCEI era chamada de NCDC. Caso o URL deste recurso mude no futuro, você pode buscar por "NCEI weather API" para encontrar o atualizado.*

## Usando a API da NCEI
Solicite seu token [aqui](https://www.ncdc.noaa.gov/cdo-web/token) e cole-o abaixo.

In [1]:
import requests


def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.

    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.

    Returns:
        A response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'rUvrEaMJLCXLPwJZOEisxChDcBjLqsvC'
        },
        params=payload
    )

**Nota: a API nos limita a 5 requisições por segundo e 10.000 requisições por dia.**

## Verificar quais conjuntos de dados estão disponíveis
Podemos fazer requisições para o endpoint `datasets` para ver quais conjuntos de dados estão disponíveis. Também passamos um dicionário no payload para obter conjuntos de dados que têm dados após a data de início de 1 de outubro de 2018.

In [2]:
response = make_request('datasets', {'startdate': '2018-10-01'})

O código de status `200` significa que tudo está OK. Mais códigos podem ser encontrados [aqui](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In [3]:
response.status_code

200

Alternativamente, podemos verificar o atributo `ok`:

In [4]:
response.ok

True

### Obter as chaves do resultado
O resultado é um payload JSON, que podemos acessar com o método `json()` do nosso objeto de resposta. Objetos JSON podem ser tratados como dicionários, então podemos acessar as chaves da mesma forma que acessaríamos um dicionário:

In [5]:
payload = response.json()
payload.keys()

dict_keys(['metadata', 'results'])

The metadata of the response will tell us information about the request and data we got back:

In [6]:
payload['metadata']

{'resultset': {'offset': 1, 'count': 11, 'limit': 25}}

### Identificar quais dados estão no resultado
A chave `results` contém os dados que solicitamos. Esta é uma lista que corresponderia às linhas no nosso dataframe. Cada entrada na lista é um dicionário, então podemos olhar para as chaves para obter os campos:

In [7]:
payload['results'][0].keys()

dict_keys(['uid', 'mindate', 'maxdate', 'name', 'datacoverage', 'id'])

### Analisar o resultado
Não queremos todos esses campos, então usaremos uma list comprehension para extrair apenas os campos `id` e `name`:

In [8]:
[(data['id'], data['name']) for data in payload['results']]

[('GHCND', 'Daily Summaries'),
 ('GSOM', 'Global Summary of the Month'),
 ('GSOY', 'Global Summary of the Year'),
 ('NEXRAD2', 'Weather Radar (Level II)'),
 ('NEXRAD3', 'Weather Radar (Level III)'),
 ('NORMAL_ANN', 'Normals Annual/Seasonal'),
 ('NORMAL_DLY', 'Normals Daily'),
 ('NORMAL_HLY', 'Normals Hourly'),
 ('NORMAL_MLY', 'Normals Monthly'),
 ('PRECIP_15', 'Precipitation 15 Minute'),
 ('PRECIP_HLY', 'Precipitation Hourly')]

## Determinar qual categoria de dados queremos
Os dados `GHCND` que contêm resumos diários são o que queremos. Agora precisamos fazer outra requisição para descobrir quais categorias de dados queremos coletar. Isso é feito no endpoint `datacategories`. Devemos passar o `datasetid` para `GHCND` no payload para que a API saiba de qual conjunto de dados estamos perguntando:

In [9]:
# get data category id
response = make_request(
    'datacategories', payload={'datasetid': 'GHCND'}
)
response.status_code

200

Como sabemos que a API nos fornece as chaves `metadata` e `results` em cada resposta, podemos verificar o que está na parte `results` do payload JSON:

In [10]:
response.json()['results']

[{'name': 'Evaporation', 'id': 'EVAP'},
 {'name': 'Land', 'id': 'LAND'},
 {'name': 'Precipitation', 'id': 'PRCP'},
 {'name': 'Sky cover & clouds', 'id': 'SKY'},
 {'name': 'Sunshine', 'id': 'SUN'},
 {'name': 'Air Temperature', 'id': 'TEMP'},
 {'name': 'Water', 'id': 'WATER'},
 {'name': 'Wind', 'id': 'WIND'},
 {'name': 'Weather Type', 'id': 'WXTYPE'}]

## Obter o ID do tipo de dado para a categoria de temperatura
Vamos trabalhar com temperaturas, então queremos a categoria de dados `TEMP`. Agora, precisamos encontrar os `datatypes` para coletar. Para isso, usamos o endpoint `datatypes` e fornecemos o `datacategoryid`, que é `TEMP`. Também especificamos um limite para o número de `datatypes` a serem retornados no payload. Se houver mais do que isso, podemos fazer outra requisição posteriormente, mas por enquanto, queremos apenas escolher alguns:

In [13]:
# get data type id
response = make_request(
    'datatypes',
    payload={
        'datacategoryid': 'TEMP',
        'limit': 100
    }
)
response.status_code

200

Podemos pegar os campos `id` e `name` para cada uma das entradas na parte `results` dos dados. Os campos que estamos interessados estão no final:

In [14]:
[(datatype['id'], datatype['name'])
 for datatype in response.json()['results']][-5:]  # look at the last 5

[('MNTM', 'Monthly mean temperature'),
 ('TAVG', 'Average Temperature.'),
 ('TMAX', 'Maximum temperature'),
 ('TMIN', 'Minimum temperature'),
 ('TOBS', 'Temperature at the time of observation')]

## Determinar qual categoria de localização queremos
Agora que sabemos quais `datatypes` iremos coletar, precisamos encontrar a localização a ser usada. Primeiro, precisamos descobrir a categoria de localização. Isso é obtido no endpoint `locationcategories`, passando o `datasetid`:

In [15]:
# get location category id
response = make_request(
    'locationcategories',
    payload={'datasetid': 'GHCND'}
)
response.status_code

200

Podemos usar `pprint` para imprimir dicionários em um formato mais fácil de ler. Depois de fazer isso, podemos ver que existem 12 categorias diferentes de localização, mas estamos interessados apenas em `CITY`:

In [16]:
import pprint
pprint.pprint(response.json())

{'metadata': {'resultset': {'count': 12, 'limit': 25, 'offset': 1}},
 'results': [{'id': 'CITY', 'name': 'City'},
             {'id': 'CLIM_DIV', 'name': 'Climate Division'},
             {'id': 'CLIM_REG', 'name': 'Climate Region'},
             {'id': 'CNTRY', 'name': 'Country'},
             {'id': 'CNTY', 'name': 'County'},
             {'id': 'HYD_ACC', 'name': 'Hydrologic Accounting Unit'},
             {'id': 'HYD_CAT', 'name': 'Hydrologic Cataloging Unit'},
             {'id': 'HYD_REG', 'name': 'Hydrologic Region'},
             {'id': 'HYD_SUB', 'name': 'Hydrologic Subregion'},
             {'id': 'ST', 'name': 'State'},
             {'id': 'US_TERR', 'name': 'US Territory'},
             {'id': 'ZIP', 'name': 'Zip Code'}]}


## Obter o ID de Localização de NYC
Para encontrar o ID de localização de Nova York, precisamos buscar todas as cidades disponíveis. Como podemos pedir à API para retornar as cidades ordenadas, podemos usar a busca binária para encontrar Nova York rapidamente sem precisar fazer muitas solicitações ou solicitar muitos dados de uma só vez. A função a seguir faz a primeira solicitação para ver o tamanho da lista e olha o primeiro valor. A partir daí, decide se precisa se mover em direção ao início ou ao final da lista comparando o item que estamos procurando com outros em ordem alfabética. Cada vez que faz uma solicitação, pode eliminar metade dos dados restantes para pesquisar.

In [17]:
def get_item(name, what, endpoint, start=1, end=None):
    """
    Grab the JSON payload for a given field by name using binary search.

    Parameters:
        - name: The item to look for.
        - what: Dictionary specifying what the item in `name` is.
        - endpoint: Where to look for the item.
        - start: The position to start at. We don't need to touch this, but the
                 function will manipulate this with recursion.
        - end: The last position of the items. Used to find the midpoint, but
               like `start` this is not something we need to worry about.

    Returns:
        Dictionary of the information for the item if found otherwise 
        an empty dictionary.
    """
    # find the midpoint which we use to cut the data in half each time
    mid = (start + (end or 1)) // 2

    # lowercase the name so this is not case-sensitive
    name = name.lower()

    # define the payload we will send with each request
    payload = {
        'datasetid': 'GHCND',
        'sortfield': 'name',
        'offset': mid,  # we will change the offset each time
        'limit': 1  # we only want one value back
    }

    # make our request adding any additional filter parameters from `what`
    response = make_request(endpoint, {**payload, **what})

    if response.ok:
        payload = response.json()

        # if response is ok, grab the end index from the response metadata the first time through
        end = end or payload['metadata']['resultset']['count']

        # grab the lowercase version of the current name
        current_name = payload['results'][0]['name'].lower()

        # if what we are searching for is in the current name, we have found our item
        if name in current_name:
            return payload['results'][0]  # return the found item
        else:
            if start >= end:
                # if our start index is greater than or equal to our end, we couldn't find it
                return {}
            elif name < current_name:
                # our name comes before the current name in the alphabet, so we search further to the left
                return get_item(name, what, endpoint, start, mid - 1)
            elif name > current_name:
                # our name comes after the current name in the alphabet, so we search further to the right
                return get_item(name, what, endpoint, mid + 1, end)
    else:
        # response wasn't ok, use code to determine why
        print(f'Response not OK, status: {response.status_code}')

Quando usamos a busca binária para encontrar Nova York, a encontramos em apenas 8 solicitações, apesar de estar próxima do meio das 1.983 entradas:

In [18]:
# get NYC id
nyc = get_item('New York', {'locationcategoryid': 'CITY'}, 'locations')
nyc

{'mindate': '1869-01-01',
 'maxdate': '2024-05-31',
 'name': 'New York, NY US',
 'datacoverage': 1,
 'id': 'CITY:US360019'}

## Obter o ID da Estação para o Central Park
Os dados mais granulares são encontrados no nível da estação:

In [20]:
central_park = get_item('NY City Central Park', {
                        'locationid': nyc['id']}, 'stations')
central_park

{'elevation': 42.7,
 'mindate': '1869-01-01',
 'maxdate': '2024-05-30',
 'latitude': 40.77898,
 'name': 'NY CITY CENTRAL PARK, NY US',
 'datacoverage': 1,
 'id': 'GHCND:USW00094728',
 'elevationUnit': 'METERS',
 'longitude': -73.96925}

## Solicitar os dados de temperatura
Finalmente, temos tudo o que precisamos para fazer nossa solicitação de dados de temperatura de Nova York. Para isso, usamos o endpoint `data` e fornecemos todos os parâmetros que coletamos durante nossa exploração da API:

In [21]:
# get NYC daily summaries data
response = make_request(
    'data',
    {
        'datasetid': 'GHCND',
        'stationid': central_park['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        # average, max, and min temperature
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'],
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

200

## Criar um DataFrame
A estação Central Park possui apenas as temperaturas mínimas e máximas diárias.

In [22]:
import pandas as pd

df = pd.DataFrame(response.json()['results'])
df.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",24.4
1,2018-10-01T00:00:00,TMIN,GHCND:USW00094728,",,W,2400",17.2
2,2018-10-02T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",25.0
3,2018-10-02T00:00:00,TMIN,GHCND:USW00094728,",,W,2400",18.3
4,2018-10-03T00:00:00,TMAX,GHCND:USW00094728,",,W,2400",23.3


Não recebemos `TAVG` porque a estação não mede essa informação.

In [23]:
df.datatype.unique()

array(['TMAX', 'TMIN'], dtype=object)

Apesar de aparecer nos dados como medindo isso... A realidade dos dados é complexa!

In [24]:
if get_item(
    'NY City Central Park', {
        'locationid': nyc['id'], 'datatypeid': 'TAVG'}, 'stations'
):
    print('Found!')

Found!


## Usando uma estação diferente
Vamos usar o aeroporto de LaGuardia. Ele contém `TAVG` (temperatura diária média):

In [26]:
laguardia = get_item(
    'LaGuardia', {'locationid': nyc['id']}, 'stations'
)
laguardia

{'elevation': 3,
 'mindate': '1939-10-07',
 'maxdate': '2024-05-31',
 'latitude': 40.77945,
 'name': 'LAGUARDIA AIRPORT, NY US',
 'datacoverage': 1,
 'id': 'GHCND:USW00014732',
 'elevationUnit': 'METERS',
 'longitude': -73.88027}

Vamos fazer nossa solicitação usando a estação do aeroporto de LaGuardia desta vez.

In [27]:
# get NYC daily summaries data
response = make_request(
    'data',
    {
        'datasetid': 'GHCND',
        'stationid': laguardia['id'],
        'locationid': nyc['id'],
        'startdate': '2018-10-01',
        'enddate': '2018-10-31',
        # temperature at time of observation, min, and max
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'],
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

200

A solicitação foi bem-sucedida, então vamos criar um DataFrame:

In [28]:
df = pd.DataFrame(response.json()['results'])
df.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",21.2
1,2018-10-01T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",25.6
2,2018-10-01T00:00:00,TMIN,GHCND:USW00014732,",,W,2400",18.3
3,2018-10-02T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",22.7
4,2018-10-02T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",26.1


Devemos verificar se recebemos o que queríamos: 31 entradas para TAVG, TMAX e TMIN (1 por dia):

In [29]:
df.datatype.value_counts()

datatype
TAVG    31
TMAX    31
TMIN    31
Name: count, dtype: int64

Salve o arquivo CSV para uso em outros notebooks.

In [26]:
df.to_csv('data/nyc_temperatures.csv', index=False)

<hr>
<div>
    <a href="./1-wide_vs_long.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./3-cleaning_data.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>